In [ ]:
# !pip install py7zr 
# !pip install transformers
# !pip install datasets
# !pip install rouge-score
# !pip install nltk
# !pip install sentencepiece

In [3]:
from transformers import AutoModel, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer

model_name = "allenai/scibert_scivocab_uncased"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████

In [ ]:
import torch.nn as nn

model.embeddings.token_type_embeddings = nn.Embedding(3,768)
# (num_modalities/num_token_types, hidden_size)
# By default, num_modalities is 2
# 768 is the dimension of the transformer

In [ ]:
import torch
import numpy as np
img_embeddings = torch.from_numpy(np.random.rand(1,10,768)).float()
tabular_embeddings=torch.from_numpy(np.random.rand(1,4,768)).float()
radiomics_embeddings=torch.from_numpy(np.random.rand(1,3,768)).float()

In [ ]:
random_embeddings=torch.cat([img_embeddings,tabular_embeddings,radiomics_embeddings],axis=1)

In [ ]:
random_embeddings.shape

torch.Size([1, 17, 768])

In [ ]:
import numpy as np
token_type_ids=torch.tensor(torch.cat([torch.zeros(1,10),torch.ones(1,4),torch.ones(1,3)*2],axis=-1),dtype=torch.int32)
position_ids=torch.tensor([range(17) for i in range(1)],dtype=torch.int32)

<ipython-input-11-1bb7ec631b9b>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token_type_ids=torch.tensor(torch.cat([torch.zeros(1,10),torch.ones(1,4),torch.ones(1,3)*2],axis=-1),dtype=torch.int32)


In [ ]:
token_type_ids

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2]],
       dtype=torch.int32)

In [ ]:
position_ids
# position_ids: Sequence only within the modality
# image: (0, ..., n_image)
# tabular: (0, ..., n_tabular)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]],
       dtype=torch.int32)

In [ ]:
output_embds=model(inputs_embeds=random_embeddings,position_ids=position_ids,token_type_ids=token_type_ids)

In [ ]:
output_embds.last_hidden_state.shape

torch.Size([1, 17, 768])

In [ ]:
output_embds.last_hidden_state

tensor([[[ 0.1345,  0.1185, -0.7807,  ...,  0.9029, -0.7225, -1.8057],
         [ 0.1493,  0.1543, -0.8277,  ...,  0.8729, -0.7316, -1.7966],
         [ 0.1243,  0.1802, -0.8694,  ...,  0.8780, -0.7229, -1.8201],
         ...,
         [-1.1394, -0.1683, -1.0218,  ..., -0.0394, -0.9424, -1.1058],
         [-0.9834, -0.1311, -1.0033,  ..., -0.0075, -1.0196, -1.1825],
         [-0.9458, -0.0936, -1.1333,  ..., -0.0198, -1.0350, -1.0842]]],
       grad_fn=<NativeLayerNormBackward0>)